In [8]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy.types
from pathlib import Path
import globals

In [9]:
current_path = Path.cwd()
data_path = current_path.parent.joinpath('data')
globals.initialize()

In [23]:
# connection parameters in globals.py

# create a connection to the PostgreSQL database
conn = psycopg2.connect(host=globals.hostname,
                        dbname=globals.database,
                        user=globals.username,
                        password=globals.pwd,
                        port=globals.port_id)


# create an engine for the connection
engine = create_engine(f'postgresql://{globals.username}:{globals.pwd}@{globals.hostname}:{globals.port_id}/{globals.database}')

#Create a cursor
cur = conn.cursor()

In [43]:
#Load CSV file into a Pandas DataFrame
df_countries = pd.read_csv(data_path.joinpath('df_countries_clean.csv'), sep=";", header=0, na_values=["",np.nan], keep_default_na=False)
display(df_countries.head())

df_cities = pd.read_csv(data_path.joinpath('df_cities_clean.csv'), sep=";", header=0, na_values=["",np.nan], keep_default_na=False)
display(df_cities.head())

df_airlines = pd.read_csv(data_path.joinpath('df_airlines_clean.csv'), sep=";", header=0, na_values=["",np.nan], keep_default_na=False)
display(df_airlines.head())

df_airports = pd.read_csv(data_path.joinpath('df_airports_clean.csv'), sep=";", header=0, na_values=["",np.nan], keep_default_na=False)
display(df_airports.head())

df_aircrafts = pd.read_csv(data_path.joinpath('df_aircrafts_clean.csv'), sep=";", header=0, na_values=["",np.nan], keep_default_na=False)
display(df_aircrafts.head())

,CountryCode,CountryName
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,"Antigua And Barbuda, Leeward Islands"
4,AI,"Anguilla, Leeward Islands"


,CityCode,CountryCode,UtcOffset,TimeZoneId,CityName
0,AAA,PF,-10:00,Pacific/Tahiti,Anaa
1,AAB,AU,+10:00,Australia/Brisbane,Arrabury
2,AAC,EG,+02:00,Africa/Cairo,El Arish
3,AAD,SO,+03:00,Africa/Mogadishu,Adado
4,AAE,DZ,+01:00,Africa/Algiers,Annaba


,AirlineID,AirlineID_ICAO,AirlineName
0,0A,GNT,Amber Air
1,0B,BMS,Blue Air
2,0D,DWT,Darwin Airline Sa
3,0J,PJZ,Premium Jet Ag
4,0K,KRT,Aircompany Kokshetau


,AirportCode,CityCode,CountryCode,LocationType,UtcOffset,TimeZoneId,Latitude,Longitude,AirportName
0,AAA,AAA,PF,Airport,-10:00,Pacific/Tahiti,-17.3525,-145.5100,Anaa
1,AAB,AAB,AU,Airport,+10:00,Australia/Brisbane,-26.6911,141.0472,Arrabury
2,AAC,AAC,EG,Airport,+02:00,Africa/Cairo,31.0733,33.8358,El Arish International
3,AAD,AAD,SO,Airport,+03:00,Africa/Mogadishu,6.0961,46.6375,Adado
4,AAE,AAE,DZ,Airport,+01:00,Africa/Algiers,36.8222,7.8092,Annaba Rabah Bitat


,AircraftCode,AirlineEquipCode,AircraftName
0,100,F100,Fokker 100
1,141,B461,BAE Systems 146-100 Passenger
2,142,B462,BAE Systems 146-200 Passenger
3,143,B463,BAE Systems 146-300 Passenger
4,14X,B461,BAE Systems 146-100 Freighter


In [42]:
cur.execute("""
DROP TABLE IF EXISTS airports;
DROP TABLE IF EXISTS cities;
DROP TABLE IF EXISTS countries;
DROP TABLE IF EXISTS airlines;
DROP TABLE IF EXISTS aircrafts;
""")

conn.commit()

In [17]:
cur.execute("""
CREATE TABLE countries (
    CountryCode char(2) NOT NULL,
    CountryName text,
    PRIMARY KEY (CountryCode)
);
CREATE TABLE cities (
    CityCode char(3) NOT NULL,
    CountryCode char(2) NOT NULL,
    UtcOffset interval,
    TimeZoneId text,
    CityName text,
    PRIMARY KEY (CityCode),
    FOREIGN KEY (CountryCode) REFERENCES countries (CountryCode)
); 
CREATE TABLE airports (
    AirportCode char(3) NOT NULL,
    CityCode char(3) NOT NULL,
    CountryCode char(2) NOT NULL,
    LocationType text,
    UtcOffset interval,
    TimeZoneId text,
    Latitude double precision,
    Longitude double precision,
    AirportName text,
    PRIMARY KEY (AirportCode),
    FOREIGN KEY (CityCode) REFERENCES cities (CityCode),
    FOREIGN KEY (CountryCode) REFERENCES countries (CountryCode)
); 
CREATE TABLE airlines (
    AirlineID char(3) NOT NULL,
    AirlineID_ICAO char(3),
    AirlineName text,
    PRIMARY KEY (AirlineID)
); 
CREATE TABLE aircrafts (
    AircraftCode char(3) NOT NULL,
    AirlineEquipCode char(4),
    AircraftName text,
    PRIMARY KEY (AircraftCode)
); 
""")

conn.commit()

In [29]:
'''
#conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
#cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)


with conn.cursor() as cursor:
    create_countries_table_(cursor)
# cur.close()
# conn.close()
'''

In [18]:
def fcn(df,table,cur):
    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

In [20]:
fcn(df_countries,'countries',cur)
fcn(df_cities,'cities',cur)
fcn(df_airports,'airports',cur)
fcn(df_airlines,'airlines',cur)
fcn(df_aircrafts,'aircrafts',cur)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [44]:
def create_airlines_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS airlines;
        CREATE TABLE airlines (
            AirlineID char(3) NOT NULL,
            AirlineID_ICAO char(3),
            AirlineName text,
            PRIMARY KEY (AirlineID)
        ); 
        """)
    
with conn.cursor() as cursor:
    create_airlines_table_(cursor)

fcn(df_airlines,'airlines',cur)

In [45]:
def create_aircrafts_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS aircrafts;
        CREATE TABLE aircrafts (
            AircraftCode char(3) NOT NULL,
            AirlineEquipCode char(4),
            AircraftName text,
            PRIMARY KEY (AircraftCode)
        ); 
        """)
    
with conn.cursor() as cursor:
    create_aircrafts_table_(cursor)

fcn(df_aircrafts,'aircrafts',cur)

In [46]:
def create_countries_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS countries;
        CREATE TABLE countries (
            CountryCode char(2) NOT NULL,
            CountryName text,
            PRIMARY KEY (CountryCode)
        );
        """)
    
with conn.cursor() as cursor:
    create_countries_table_(cursor)

fcn(df_countries,'countries',cur)

In [47]:
def create_cities_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS cities;
        CREATE TABLE cities (
            CityCode char(3) NOT NULL,
            CountryCode char(2) NOT NULL,
            UtcOffset interval,
            TimeZoneId text,
            CityName text,
            PRIMARY KEY (CityCode),
            FOREIGN KEY (CountryCode) REFERENCES countries (CountryCode)
        ); 
        """)
    
with conn.cursor() as cursor:
    create_cities_table_(cursor)

fcn(df_cities,'cities',cur)

In [48]:
def create_airports_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS airports;
        CREATE TABLE airports (
            AirportCode char(3) NOT NULL,
            CityCode char(3) NOT NULL,
            CountryCode char(2) NOT NULL,
            LocationType text,
            UtcOffset interval,
            TimeZoneId text,
            Latitude double precision,
            Longitude double precision,
            AirportName text,
            PRIMARY KEY (AirportCode),
            FOREIGN KEY (CountryCode) REFERENCES countries (CountryCode),
            FOREIGN KEY (CityCode) REFERENCES cities (CityCode)
        ); 
        """)
    
with conn.cursor() as cursor:
    create_airports_table_(cursor)

fcn(df_airports,'airports',cur)

In [41]:
conn.commit()

In [22]:
#Close the cursor and the connection
cur.close()
conn.close()